In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.insert(0, '../..')

import os
from src.images.datasets import get_denta_labels, get_dynamic_features
from src.images.preprocess import extract_faces_denta, extract_landmarks
from src.images.detector import get_face_detector
import src.images.utils as utils


# Features and estimators

Load dynamic features first.

In [2]:
dynamic_data = get_dynamic_features('../../data/denta_v1/ex3_data.csv')

In [3]:
from src.images.features import (
    KarolewskiFilteredFeaturesExtractor,
    EmptyFeaturesExtractor
)
from src.images.method_experiments import estimators

features = [
    {
        'name': 'Dynamic',
        'cls': EmptyFeaturesExtractor,
        'extra_features': dynamic_data,
        'add_extra_features': True,
    },
    {
        'name': 'KarolewskiFiltered+Dynamic',
        'cls': KarolewskiFilteredFeaturesExtractor,
        'extra_features': dynamic_data,
        'add_extra_features': True,
    },
    {
        'name': 'KarolewskiFiltered',
        'cls': KarolewskiFilteredFeaturesExtractor
    },
]

# Denta

In [4]:
denta_df = get_denta_labels('../../data/denta_v1', split_type='random')
denta_data_path = os.path.join('..', '..', 'experiments', 'images_results')


In [8]:
app = get_face_detector()
extract_faces_denta('../../data/denta_v1', denta_df, detector=lambda image: app.get(image))
extract_landmarks('../../data/denta_v1', denta_df)


100%|██████████| 258/258 [00:27<00:00,  9.51it/s]
Lightning automatically upgraded your loaded checkpoint from v1.5.10 to v2.0.4. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file d:\Users\micha\Documents\Side projects\tmd-recognition\data\snapshots\WFLW\pip_32_16_60_r101_l2_l1_10_1_nb10.ckpt`
100%|██████████| 258/258 [00:50<00:00,  5.10it/s]


## Features preparation

In [10]:
from src.images.method_experiments import prepare_features

features_denta_0_32 = prepare_features('../../data/denta_v1', features, df=denta_df, reference_points=(0, 32))
features_denta_64_68 = prepare_features('../../data/denta_v1', features, df=denta_df, reference_points=(64, 68))

100%|██████████| 258/258 [00:00<00:00, 351.02it/s]


## Aggregation tests

In [ ]:
from src.images.method_experiments import test_concat_vs_avg

agg_tests = test_concat_vs_avg(estimators, features_denta_64_68, cv=5, stratified=True, iterations=40)
utils.save_pickle(os.path.join(denta_data_path, 'aggregation_5fold_64_68.pkl'), agg_tests)

In [19]:
from src.images.method_experiments import test_concat_vs_avg

agg_tests = test_concat_vs_avg(estimators, features_denta_0_32, cv=5, stratified=True, iterations=40)
utils.save_pickle(os.path.join(denta_data_path, 'aggregation_5fold_0_32.pkl'), agg_tests)

100%|██████████| 1440/1440 [01:39<00:00, 14.46it/s]


In [21]:
from src.images.method_experiments import test_concat_vs_avg, dummy_estimators

agg_tests = test_concat_vs_avg(dummy_estimators, features_denta_0_32, cv=5, stratified=True, iterations=50)
utils.save_pickle(os.path.join(denta_data_path, 'dummy_aggregation_5fold.pkl'), agg_tests)

100%|██████████| 1800/1800 [00:12<00:00, 146.15it/s]
